#Module de creation de vocabulaire

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import unicodedata as uni
import re
import time
def normalize(string):
    return uni.normalize('NFKD',string).encode('ascii','ignore')
from sys import stdout
import sys

In [22]:
TRAINING_SET_PATH = 'Data/cleantrainV4.csv'
TEST_SET_PATH = 'Data/cleantestV4.csv'
SEPARATOR = ','
INDEX_COL = 'Identifiant_Produit'

CAT_COL = 'Categorie3'
DESC_COL = 'Description'
LIBE_COL = 'Libelle'
MARQ_COL = 'Marque'
PRICE_COL = 'Prix'
CDIS_COL = 'Produit_Cdiscount'
MARQ_DESC = 'Marque_in_Description' 
MARQ_LIBE = 'Marque_in_Libelle'
INT_LIBE_DESC = 'Intersection_Libe_Desc'
LEN_LIBE = 'Longueur_Libelle'
LEN_DESC = 'Longueur_Description'
LEN_INT = 'Longueur Intersection'
LIBE_DESC = 'Percent_Libe_in_Desc'

PERCENT_CAPTURE_LIBE = 'Percent_capture_libe'
PERCENT_CAPTURE_DESC = 'Percent_capture_desc'
PERCENT_CAPTURE_INT = 'Percent_capture_int'

MIN_DF_BY_CATEGORY_DESC = 0.02
MIN_DF_BY_CATEGORY_LIBE = 0.01
MIN_DF_BY_CATEGORY_INT = 0.01

NGRAM_RANGE_LIBE = (1,1)
NGRAM_RANGE_DESC = (1,1)
NGRAM_RANGE_INT = (1,1)

VOCABULARY_PATH_DESC = 'Data/vocabularyDesc'
VOCABULARY_PATH_LIBE = 'Data/vocabularyLibe'
VOCABULARY_PATH_INT = 'Data/vocabularyInt'

SAVE_TRAINING_PATH = 'Data/cleantrain.csv'
SAVE_TEST_PATH = 'Data/cleantest.csv'

In [17]:
print "loading Data"
Ctrain = pd.read_csv(TRAINING_SET_PATH, sep=SEPARATOR,index_col=INDEX_COL)
Ctrain.fillna(' ',inplace=True)
Ctest = pd.read_csv(TEST_SET_PATH, sep=SEPARATOR,index_col=INDEX_COL)
Ctest.fillna(' ',inplace=True)

loading Data


Price Cleaning try to remove some of the errors.

get both bigram and single word

In [1]:
#print "get both bigram and single word"
#cuts = np.r_[np.arange(0,15000001,1000000),[15786885]]
#for i in range(cuts.shape[0]-1):
#    Ctrain.ix[cuts[i]:cuts[i+1]-1,DESC_COL] = Ctrain.ix[cuts[i]:cuts[i+1]-1,DESC_COL].apply(lambda x: " ".join([word + " " + " ".join(word.split("_"))  if "_" in word else word for word in x.split()]))
#    stdout.write("\r\x1b[K" + "[" + (i+1) * "--" + (16-(i+1))*"  " + "] %.2f" %((i+1)*6.25))
#    stdout.flush()
#Ctest[AUGMENTED_DESC_COL] = Ctest[AUGMENTED_DESC_COL].apply(lambda x: " ".join([word + " " + " ".join(word.split("_"))  if "_" in word else word for word in x.split()]))

Create a vocabulary by categories by Columns LIBELLE DESCRIPTION MARQUE INT

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

def create_vocabulary(voc_col,min_df,ngram_range):
    Counts = {}
    categories = set(Ctrain[CAT_COL].unique())
    length = len(categories)
    print "creating vocabulary For " + voc_col
    t = time.time()
    progress = 0
    for cat in categories:
        try:
            Counts[cat] = CountVectorizer(min_df=min_df,ngram_range = ngram_range)
            subtrain = Ctrain[Ctrain[CAT_COL] == cat]
            Counts[cat].fit(subtrain[voc_col])
        except:
            Counts[cat] = CountVectorizer(min_df=min_df,ngram_range = ngram_range)
            Counts[cat].fit(['empty','empty'])
        progress +=1
        percent = progress * 100.0/ length
        count = int(np.floor(percent/2))
        elapse_t = time.time() - t
        eta = elapse_t * (100.0 - percent) / percent / 60
        stdout.write("\r\x1b[K" + "[" + count * "-" + (50-count)*" " + "] %.1f p" %percent + "       eta :  %.1f min" %eta )
        stdout.flush()
    elapse_t = (time.time() - t) / 60
    print " "
    print "done in %.1f min" %elapse_t
    
    vocabulary = set()
    print "merging vocabulary"
    t = time.time()
    progress = 0
    for cat in categories:
        vocabulary = vocabulary.union(set(Counts[cat].vocabulary_.keys()))
        progress +=1
        percent = progress * 100.0/ length
        count = int(np.floor(percent/2))
        elapse_t = time.time() - t
        eta = elapse_t * (100.0 - percent) / percent / 60
        stdout.write("\r\x1b[K" + "[" + count * "-" + (50-count)*" " + "] %.1f p" %percent + "       eta :  %.1f min" %eta )
        stdout.flush()
    elapse_t = (time.time() - t) / 60
    print " "
    print "done in %.1f min" % elapse_t
    print "vocabulary length %d words" % len(vocabulary)
    return vocabulary


In [24]:
vocabularyDESC = create_vocabulary(DESC_COL,MIN_DF_BY_CATEGORY_DESC,NGRAM_RANGE_DESC)

creating vocabulary For Description
[--------------------------------------------------] 100.0 p       eta :  0.0 min 
done in 26.7 min
merging vocabulary
[--------------------------------------------------] 100.0 p       eta :  0.0 min 
done in 0.3 min
vocabulary length 91848 words


In [25]:
vocabularyLIBE = create_vocabulary(LIBE_COL,MIN_DF_BY_CATEGORY_LIBE,NGRAM_RANGE_LIBE)

creating vocabulary For Libelle
[--------------------------------------------------] 100.0 p       eta :  0.0 min 
done in 25.5 min
merging vocabulary
[--------------------------------------------------] 100.0 p       eta :  0.0 min 
done in 0.2 min
vocabulary length 76166 words


In [26]:
vocabularyINT = create_vocabulary(INT_LIBE_DESC,MIN_DF_BY_CATEGORY_INT,NGRAM_RANGE_INT)

creating vocabulary For Intersection_Libe_Desc
[--------------------------------------------------] 100.0 p       eta :  0.0 min 
done in 24.9 min
merging vocabulary
[--------------------------------------------------] 100.0 p       eta :  0.0 min 
done in 0.2 min
vocabulary length 56872 words


Count word and compute word catching by document.

In [38]:
def insert_percent(voc,colin,colout):
    newcount = CountVectorizer(vocabulary=voc)
    vect = newcount.transform(Ctrain[colin])
    s = vect.sum(axis=1).flatten()
    Ctrain['temp'] = s.T
    cuts = np.r_[np.arange(0,15000001,1000000),[15786885]]
    for i in range(cuts.shape[0]-1):
        Ctrain.ix[cuts[i]:cuts[i+1]-1,colout] = Ctrain.ix[cuts[i]:cuts[i+1]-1,[colin,'temp']].apply(lambda x: (x['temp']+1)*1.0/(len(x[colin].split())+1),axis=1)
        stdout.write("\r\x1b[K" + "[" + (i+1) * "--" + (16-(i+1))*"  " + "] %.2f" %((i+1)*6.25))
        stdout.flush()
    Ctrain.drop('temp',axis=1,inplace=True)
    vect2 = newcount.transform(Ctest[colin])
    s2 = vect2.sum(axis=1).flatten()
    Ctest['temp'] = s2.T
    Ctest[colout] = Ctest[[colin,'temp']].apply(lambda x: (x['temp']+1)*1.0/(len(x[colin].split())+1),axis=1)
    Ctest.drop('temp',axis=1,inplace=True)
    print ' '
    print 'Done'

In [39]:
print "Description percent"
insert_percent(vocabularyDESC,DESC_COL,PERCENT_CAPTURE_DESC)
print "Libelle percent"
insert_percent(vocabularyLIBE,LIBE_COL,PERCENT_CAPTURE_LIBE)
print "Intersection percent"
insert_percent(vocabularyINT,INT_LIBE_DESC,PERCENT_CAPTURE_INT)

Description percent
[--------------------------------] 100.00 
Done
Libelle percent
[--------------------------------] 100.00 
Done
intersection percent
[--------------------------------] 100.00 
Done


Save results for next step.

In [40]:
np.save(VOCABULARY_PATH_DESC,np.array(list(vocabularyDESC)))
np.save(VOCABULARY_PATH_LIBE,np.array(list(vocabularyLIBE)))
np.save(VOCABULARY_PATH_INT ,np.array(list(vocabularyINT)))

Ctrain.to_csv(SAVE_TRAINING_PATH)
Ctest.to_csv(SAVE_TEST_PATH)

Execution time : 1h30 for 1gram

Execution time : 2h30 for 1-2gram